In [1]:
import numpy as np 
import pandas as pd

In [2]:
news = pd.read_csv('/notebooks/notebooks/data.csv', sep=',')
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 500
#DOCUMENT="description"
#TOPIC="title"

#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

In [3]:
subset_news

,Query,Intermediate MQL,Reasoning
0,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
1,Query,Intermediate MQL,NaN
2,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
3,Query,Intermediate MQL,NaN
4,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
5,Query,Intermediate MQL,NaN
6,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,NaN


In [4]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

ModuleNotFoundError: No module named 'langchain'

In [6]:
#!pip install langchain